In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys

from urllib.request import urlretrieve

dir_data='/Users/ahenny/'

In [ ]:
yrs=np.arange(2019,2023,1)
months=np.arange(1,13,1)
month_strs=['01','02','03','04','05','06','07','08','09','10','11','12']
days_month=[31,28,31,30,31,30,31,31,30,31,30,31]
for i in range(len(yrs)):
    year=yrs[i]
    days_month[1]=28
    if year%4==0:
        days_month[1]=29
        
    for j in range(len(month_strs)):
        month_str=month_strs[j]
        for k in range(days_month[j]):
            if k<9:
                day_str='0'+str(k+1)
            else:
                day_str=str(k+1)

            filename='/Users/ahenny/persiann_cdr_'+str(year)+month_str+day_str+'.nc'
            print(filename)
            
            if year<2013:
                urlretrieve('https://www.ncei.noaa.gov/data/precipitation-persiann/access/'+str(year)+'/PERSIANN-CDR_v01r01_'+str(year)+month_str+day_str+'_c20140523.nc',filename)
            else:
                if year==2013:
                    strs=['c20140908']
                elif year==2014:
                    strs=['c20140908','c20150330']
                elif year==2015:
                    strs=['c20150728','c20151028','c20160324']
                elif year==2016:
                    strs=['c20160720','c20161115','c20170119','c20170930']
                elif year==2017:
                    strs=['c20170930','c20171211','c20180801','c20180327']
                elif year==2018:
                    strs=['c20181102','c20190204','c20190509']
                elif year==2019:
                    strs=['c20190806','c20190923','c20200214','c20200522']
                elif year==2020:
                    strs=['c20200810','c20210208','c20210504']
                elif year==2021:
                    strs=['c20210625','c20211124','c20220524','c20220415']
                elif year==2022:
                    strs=['c20220819','c20220909','c20230111','c20230406']

                success='no'
                count=0
                str_sel=strs[count]
                while success=='no':
                    try:
                        urlretrieve('https://www.ncei.noaa.gov/data/precipitation-persiann/access/'+str(year)+'/PERSIANN-CDR_v01r01_'+str(year)+month_str+day_str+'_'+str_sel+'.nc',filename)
                        success='yes'
                    except:
                        count=count+1
                        str_sel=strs[count]
                            
                        
                if success=='no':
                    print(filename+' MISSED')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

yrs=np.arange(1983,2023,1)
months=np.arange(1,13,1)
month_strs=['01','02','03','04','05','06','07','08','09','10','11','12']
days_month=[31,28,31,30,31,30,31,31,30,31,30,31]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    days_month[1]=28
    if year%4==0:
        days_month[1]=29
        
    for j in range(len(month_strs)):
        print(j)
        month_str=month_strs[j]
        for k in range(days_month[j]):
            if k<9:
                day_str='0'+str(k+1)
            else:
                day_str=str(k+1)

            filename='/Users/ahenny/persiann_cdr_'+str(year)+month_str+day_str+'.nc'
            ds=xr.open_dataset(filename)
            precip=ds['precipitation']
            precip=precip.transpose('time','lat','lon')
            #precip=precip.where(precip>=0)
            
            if j==0 and k==0:
                precip_concat=precip
            else:
                precip_concat=xr.concat([precip_concat,precip],dim='time')
                
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time
    dk.coords['lat']=precip_concat.lat
    dk.coords['lon']=precip_concat.lon
    
    dk['precip'].attrs["standard_name"]='precipitation_amount'
    dk['precip'].attrs["long_name"]='NOAA Climate Data Record of PERSIANN-CDR daily precipitation'
    dk['precip'].attrs["units"]='mm'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    
    try:
        os.remove(dir_data+'persiann_cdr_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'persiann_cdr_'+str(year)+'.nc',mode='w',format='NETCDF4')
    
    ds.close()
    precip.close()
    precip_concat.close()
    dk.close()

In [ ]:
#Visualize missing data
ds=xr.open_dataset(dir_data+'persiann_cdr_'+str(1984)+'.nc')
defined=ds['precip'][54,:,:]
defined=defined.where(defined>=0)

import matplotlib.pyplot as plt
fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1)
test=precip.transpose('time','lat','lon')
test=test.where(test>=0)
ax.pcolormesh(defined.lon.values,defined.lat.values,defined.values,vmin=0,vmax=50,cmap=plt.cm.RdYlGn_r)
plt.show()